### 爬虫一级页面

可以直接用浏览器插件，现在发现两个比较好用的：

1.easy scraper 2.instant data scraper

### 爬详情页

用easyspider软件（易采集）：https://www.easyspider.net/index_english.html

教程：https://space.bilibili.com/3494372528097534/video?tid=0&special_type=&pn=1&keyword=&order=pubdate

**自用教程：**

1. 操作路径：python--easyspider folder--点击带羊角application

2. 操作详情：

- 创建任务：中文--设计/修改任务--使用带用户信息浏览器设计（需要登录的就选这个）--开始设计（无需改路径）-- 弹出浏览器-- 创建新任务 or 修改任务

- 设计任务（关键性操作）：

    - 爬虫时登录态丢失：重新修改任务，打开网页选项右键“试运行”，重新走一遍登录流程

    - xpath定位不准：

        - 解决方法1：用类名来定位: //DIV[@class='desc']  类名查看方式：（F12，点击最左上角的方框加箭头按钮，选择元素，自动定位类名）

        - 解决方法2：点击按钮：查看其他等价的xpath

    - 分页：

        - 分页按钮：教程：https://www.bilibili.com/video/BV1th411A7ey/?spm_id_from=333.788&vd_source=0124fc64be500ed32c4563414df3cddb

        - 滚动加载：打开网页选项中设置：“打开网页后是否向下滚动”

    - 提取元素：
    
        - 采集一个字段：右键点击某元素--再右键点击同类元素--选中全部--采集数据 or 循环点击每个元素
        
        - 采集多个字段：右键点击某元素--采集该元素文本--右键点击其他元素--采集数据


- 执行任务：保存任务后--返回首页--点击：查看/管理/执行任务--任务信息--执行任务--本地直接执行（带用户信息模式）--执行后，需要等待程序自动删除TempUserDataFolder中的文件，否则需要手动删--数据在Data文件夹中查看



### python代码爬

1.更改网页url

2.修改所有类名（F12，点击最左上角的方框加箭头按钮，选择元素，自动定位类名）

In [6]:
# 这个代码只能爬取无需登录页面的基础信息（即不在列表中的元素），但至少能成功跑出来

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

# 初始化 WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

try:
    # 访问网页
    driver.get("https://hotels.ctrip.com/hotels/detail/?hotelId=42110698&checkIn=2024-04-21&checkOut=2024-04-22&cityId=2&minprice=&mincurr=&adult=1&children=0&ages=&crn=1&curr=&fgt=&stand=&stdcode=&hpaopts=&mproom=&ouid=&shoppingid=&roomkey=&highprice=-1&lowprice=0&showtotalamt=&hotelUniqueKey=")
    

    # 提取基础单个数据
    hotel_titles = [el.text for el in driver.find_elements(By.CLASS_NAME, "detail-headline_name")] # 修改这里的类名
    hotel_ratings = [el.text for el in driver.find_elements(By.CLASS_NAME, "detail-headreview_score_value")] # 修改这里的类名
    hotel_desc = [el.text for el in driver.find_elements(By.CLASS_NAME, "detail-headline_desc_text")] # 修改这里的类名

    # 创建 DataFrame 并保存到 Excel
    data = {
        "Title": hotel_titles,
        "Rating": hotel_ratings,
        "Desc": hotel_desc,

    }
    df = pd.DataFrame(data)
    df.to_excel("hotels_data.xlsx", index=False)
    print('文件已成功保存')

except Exception as e:
    print(f"在处理中发生错误：{e}")

finally:
    # 关闭浏览器
    driver.quit()


文件已成功保存


以下代码还不成功，跑不出列表中的元素信息

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

# 初始化 WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

try:
    # 访问网页
    driver.get("https://hotels.ctrip.com/hotels/detail/?hotelId=36716510&checkIn=2024-04-21&checkOut=2024-04-22&cityId=2&minprice=&mincurr=&adult=1&children=0&ages=&crn=1&curr=&fgt=&stand=&stdcode=&hpaopts=&mproom=&ouid=&shoppingid=&roomkey=&poicode=13775&latitude=31.14128&longitude=121.668045&poiname=%E8%BF%AA%E5%A3%AB%E5%B0%BC%E5%BA%A6%E5%81%87%E5%8C%BA&highprice=-1&lowprice=0&showtotalamt=&hotelUniqueKey=")
        

    # 提取酒店基本信息
    hotel_title = driver.find_element(By.CLASS_NAME, "detail-headline_name").text  # 假设只有一个标题
    hotel_rating = driver.find_element(By.CLASS_NAME, "detail-headreview_score_value").text
    hotel_desc = driver.find_element(By.CLASS_NAME, "detail-headline_desc_text").text

    # 提取房间信息
    cards = driver.find_elements(By.CSS_SELECTOR, ".roomlist .roomcard")
    rooms = []
    for card in cards:
        room_title = card.find_element(By.CSS_SELECTOR, ".roomname").text
        room_price = card.find_element(By.CSS_SELECTOR, ".price-display .price-display-hover").text
        rooms.append({"Title": room_title, "Price": room_price})

    # 将酒店信息保存为 DataFrame
    hotel_data = {
        "Title": [hotel_title],
        "Rating": [hotel_rating],
        "Desc": [hotel_desc]
    }
    hotel_df = pd.DataFrame(hotel_data)
    hotel_df.to_excel("hotel_info.xlsx", index=False)

    # 将房间信息保存为另一个 DataFrame
    rooms_df = pd.DataFrame(rooms)
    rooms_df.to_excel("room_info.xlsx", index=False)

    print('文件已成功保存')

except Exception as e:
    print(f"在处理中发生错误：{e}")

finally:
    # 关闭浏览器
    driver.quit()

文件已成功保存


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 初始化 WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

try:
    # 访问网页
    driver.get("https://hotels.ctrip.com/hotels/detail/?hotelId=36716510&checkIn=2024-04-21&checkOut=2024-04-22&cityId=2&minprice=&mincurr=&adult=1&children=0&ages=&crn=1&curr=&fgt=&stand=&stdcode=&hpaopts=&mproom=&ouid=&shoppingid=&roomkey=&poicode=13775&latitude=31.14128&longitude=121.668045&poiname=%E8%BF%AA%E5%A3%AB%E5%B0%BC%E5%BA%A6%E5%81%87%E5%8C%BA&highprice=-1&lowprice=0&showtotalamt=&hotelUniqueKey=")
    
    # 等待网页加载，确保所有内容都被加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".roomcard"))
    )

    # 提取房间信息
    cards = driver.find_elements(By.CSS_SELECTOR, ".roomcard")
    print(f"找到 {len(cards)} 个房间卡片")  # 打印房间卡片的数量

    rooms = []
    for card in cards:
        room_title = card.find_element(By.CSS_SELECTOR, ".roomname").text
        room_price = card.find_element(By.CSS_SELECTOR, ".price-display.price-display-hover").text
        rooms.append({"Title": room_title, "Price": room_price})
        print(f"Title: {room_title}, Price: {room_price}")  # 打印每个房间的标题和价格

    # 将房间信息保存为 DataFrame
    if rooms:
        rooms_df = pd.DataFrame(rooms)
        rooms_df.to_excel("room_info.xlsx", index=False)
        print('文件已成功保存')
    else:
        print("没有找到任何房间数据。")

except Exception as e:
    print(f"在处理中发生错误：{e}")

finally:
    # 关闭浏览器
    driver.quit()


在处理中发生错误：Message: 

